<a href="https://colab.research.google.com/github/fred-ykv/Value-Investing-In-Python/blob/master/Web_Scrap_Finviz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen

pd.set_option('display.max_colwidth', 25)

# Entrada do Ticker

In [87]:
symbol = input('Enter a ticker: ')
print ('Getting data for ' + symbol + '...\n')

Enter a ticker: wire
Getting data for wire...



# Set up scraper

In [88]:
url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

In [89]:
def get_fundamentals():
    try:
        # Find fundamentals table
        fundamentals = pd.read_html(str(html), attrs = {'class': 'snapshot-table2'})[0]
        
        # Clean up fundamentals dataframe
        fundamentals.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
        colOne = []
        colLength = len(fundamentals)
        for k in np.arange(0, colLength, 2):
            colOne.append(fundamentals[f'{k}'])
        attrs = pd.concat(colOne, ignore_index=True)
    
        colTwo = []
        colLength = len(fundamentals)
        for k in np.arange(1, colLength, 2):
            colTwo.append(fundamentals[f'{k}'])
        vals = pd.concat(colTwo, ignore_index=True)
        
        fundamentals = pd.DataFrame()
        fundamentals['Attributes'] = attrs
        fundamentals['Values'] = vals
        fundamentals = fundamentals.set_index('Attributes')
        return fundamentals

    except Exception as e:
        return e

In [90]:
def get_news():
    try:
        # Find news table
        news = pd.read_html(str(html), attrs = {'class': 'fullview-news-outer'})[0]
        links = []
        for a in html.find_all('a', class_="tab-link-news"):
            links.append(a['href'])
        
        # Clean up news dataframe
        news.columns = ['Date', 'News Headline']
        news['Article Link'] = links
        news = news.set_index('Date')
        return news

    except Exception as e:
        return e

In [91]:
def get_insider():
    try:
        # Find insider table
        insider = pd.read_html(str(html), attrs = {'class': 'body-table'})[0]
        
        # Clean up insider dataframe
        insider = insider.iloc[1:]
        insider.columns = ['Trader', 'Relationship', 'Date', 'Transaction', 'Cost', '# Shares', 'Value ($)', '# Shares Total', 'SEC Form 4']
        insider = insider[['Date', 'Trader', 'Relationship', 'Transaction', 'Cost', '# Shares', 'Value ($)', '# Shares Total', 'SEC Form 4']]
        insider = insider.set_index('Date')
        return insider

    except Exception as e:
        return e

In [92]:
print ('Fundamental Ratios: ')
print(get_fundamentals())

Fundamental Ratios: 
                 Values
Attributes             
Index                 -
Market Cap        3.63B
Income          717.80M
Sales             3.02B
Book/sh           99.60
...                 ...
ATR                7.43
Volatility  5.82% 3.59%
Prev Close       199.61
Price            192.39
Change           -3.62%

[72 rows x 1 columns]


In [93]:
print ('\nRecent News: ')
print(get_news())


Recent News: 
Length of values (100) does not match length of index (103)


In [94]:
print ('\nRecent Insider Trades: ')
print(get_insider())


Recent Insider Trades: 
                 Trader Relationship      Transaction    Cost # Shares  \
Date                                                                     
Feb 21  Ford Matthew D.   Controller             Sale  200.00     2725   
Feb 17  Ford Matthew D.   Controller             Sale  200.00      275   
Jul 21  Ford Matthew D.   Controller  Option Exercise   51.63     1000   

       Value ($) # Shares Total       SEC Form 4  
Date                                              
Feb 21    545000          34138  Feb 21 12:30 PM  
Feb 17     55000          36863  Feb 21 12:30 PM  
Jul 21     51630          31500  Jul 22 12:51 PM  


In [95]:
print ('\nFundamental Ratios: ')
print(get_fundamentals())

print ('\nRecent News: ')
print(get_news())

print ('\nRecent Insider Trades: ')
print(get_insider())


Fundamental Ratios: 
                 Values
Attributes             
Index                 -
Market Cap        3.63B
Income          717.80M
Sales             3.02B
Book/sh           99.60
...                 ...
ATR                7.43
Volatility  5.82% 3.59%
Prev Close       199.61
Price            192.39
Change           -3.62%

[72 rows x 1 columns]

Recent News: 
Length of values (100) does not match length of index (103)

Recent Insider Trades: 
                 Trader Relationship      Transaction    Cost # Shares  \
Date                                                                     
Feb 21  Ford Matthew D.   Controller             Sale  200.00     2725   
Feb 17  Ford Matthew D.   Controller             Sale  200.00      275   
Jul 21  Ford Matthew D.   Controller  Option Exercise   51.63     1000   

       Value ($) # Shares Total       SEC Form 4  
Date                                              
Feb 21    545000          34138  Feb 21 12:30 PM  
Feb 17     55000   

In [96]:
print ('Fundamental Ratios: ')
fundamentals = get_fundamentals()
if 'Market Cap' in fundamentals.index:
    print(fundamentals.loc['Market Cap'])
else:
    print('Market Cap is not listed.')

Fundamental Ratios: 
Values    3.63B
Name: Market Cap, dtype: object


In [97]:
market_cap_str = fundamentals.loc['Market Cap']['Values']
market_cap_str = market_cap_str.strip()

if market_cap_str[-1] == 'B':
    market_cap = float(market_cap_str[:-1]) * 1000000000
elif market_cap_str[-1] == 'M':
    market_cap = float(market_cap_str[:-1]) * 1000000
else:
    market_cap = float(market_cap_str)

print(f'Market Cap: {int(market_cap)}')

Market Cap: 3630000000


In [98]:
roi_str = fundamentals.loc['ROI']['Values']
roi_str = roi_str.replace('%', '')
roi = float(roi_str)
roi_perc = roi/100
roi_perc_str = str(roi_perc).replace('.', ',')

print(f'ROI: {roi_perc_str}')

ROI: 0,389


In [99]:
roe_str = fundamentals.loc['ROE']['Values']
roe_str = roe_str.replace('%', '')
roe = float(roe_str)
roe_perc = roe/100
roe_perc_str = str(roe_perc).replace('.', ',')

print(f'ROE: {roe_perc_str}')

ROE: 0,44


In [100]:
payout_str = fundamentals.loc['Payout']['Values']
payout_str = payout_str.replace('%', '')
payout = float(payout_str)
payout_perc = payout/100
payout_perc_str = str(payout_perc).replace('.', ',')

print(f'Payout: {payout_perc_str}')

Payout: 0,002


In [101]:
div_str = fundamentals.loc['Dividend %']['Values']
div_str = div_str.replace('%', '')
div = float(div_str)
div_perc = div/100
div_perc_str = str(div_perc).replace('.', ',')

print(f'Dividend %: {div_perc_str}')

Dividend %: 0,0004


In [122]:
epsn5_str = fundamentals.loc['EPS next 5Y']['Values']
epsn5_str = epsn5_str.replace('%', '')
epsn5 = float(epsn5_str)
epsn5_perc = epsn5/100
epsn5_perc_str = str(epsn5_perc).replace('.', ',')

print(f'EPS next 5: {epsn5_perc_str}')

EPS next 5: 0,1


In [102]:
eps_str = fundamentals.loc['EPS (ttm)']['Values']
eps = float(eps_str)
eps_str = str(eps).replace('.', ',')

print(f'EPS: {eps_str}')

EPS: 36,91


In [103]:
LPA_str = fundamentals.loc['Book/sh']['Values']
LPA = float(LPA_str)
LPA_str = str(LPA).replace('.', ',')

print(f'LPA: {LPA}')


LPA: 99.6


In [104]:
import math

eps = float(eps)
LPA = float(LPA)

# Calcular o valor intrínseco em dólar
valor_intrinseco = math.sqrt(eps * LPA * 22.5)

print(f"O valor intrínseco da ação é de ${valor_intrinseco:.2f}")


O valor intrínseco da ação é de $287.60


In [105]:
 price_str = fundamentals.loc['Price']['Values']
 price = float(price_str)
 price_str = str(price).replace('.', ',')

print(f'Preço Atual: {price_str}')

Preço Atual: 192,39


In [106]:
# Calcular a diferença entre o valor intrínseco e o preço atual
diferenca = valor_intrinseco - price

# Calcular a diferença percentual
dif_percentual = diferenca / valor_intrinseco * 100

# Verificar se está com ágio ou deságio
if price > valor_intrinseco:
    agio_desagio = 'ágio'
else:
    agio_desagio = 'deságio'

# Exibir os resultados
print(f"Valor intrínseco: ${valor_intrinseco:.2f}")
print(f"Preço atual: ${price:.2f}")
print(f"Diferença: ${diferenca:.2f}")
print(f"Diferença percentual: {dif_percentual:.2f}% {agio_desagio}")

Valor intrínseco: $287.60
Preço atual: $192.39
Diferença: $95.21
Diferença percentual: 33.11% deságio


In [107]:
beta_str = fundamentals.loc['Beta']['Values']
beta = float(beta_str)
beta_str = str(beta).replace('.', ',')

print(f'Beta: {beta_str}')

Beta: 1,36


In [108]:
tax_reinvest_ll = 1 - payout_perc
tax_reinvest_ll_str = str(tax_reinvest_ll).replace('.', ',')

print(f'Taxa de Reinvestimento LL: {tax_reinvest_ll_str}')

Taxa de Reinvestimento LL: 0,998


In [109]:
roe_str = fundamentals.loc['ROE']['Values']
roe_str = roe_str.replace('%', '')
roe = float(roe_str)
roe_perc = roe/100

gll = tax_reinvest_ll*roe_perc
gll_str = str(gll).replace('.', ',')

print(f'gLL: {gll_str}')

gLL: 0,43912


In [110]:
gain_leverage = roe_perc - roi_perc
gain_leverage_str = str(gain_leverage).replace('.', ',')

print(f'Ganho de Alavancagem: {gain_leverage_str}')

Ganho de Alavancagem: 0,05099999999999999


In [111]:
gaf = roe_perc/roi_perc
gaf_str = str(gaf).replace('.', ',')

print(f'Grau de Alavancagem Financeira: {gaf_str}')

Grau de Alavancagem Financeira: 1,1311053984575836


In [112]:

# Requisição Url
url = "https://ycharts.com/indicators/10_year_treasury_rate"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

# Parser
html = soup(webpage, "html.parser")

# Buscando Valor
xpath = '/html/body/main/div/div[2]/div/div/div[2]'
value_elem = html.find('div', {'class':'key-stat-title'}).get_text()

if value_elem is not None:
    value = value_elem.strip()
    value = value.split(' ')[0] # Removendo Strings
    value = value.replace('%', '')

    # Removendo Caracteres
    Rf = float(value.replace(',', '.'))
    Rf_str = str(Rf).replace('.', ',') + '%' # Corrigindo Erro

    print(f'Taxa de Juros Livre de Risco (Rf): {Rf_str}')

Taxa de Juros Livre de Risco (Rf): 3,82%


In [113]:

# Requisição Url
url = "https://ycharts.com/indicators/us_consumer_price_index_yoy"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

# Parser
html = soup(webpage, "html.parser")

# Buscando Valor
xpath = '/html/body/main/div/div[2]/div/div/div[2]'
value_elem = html.find('div', {'class':'key-stat-title'}).get_text()

if value_elem is not None:
    value = value_elem.strip()
    value = value.split(' ')[0] # Removendo Strings
    value = value.replace('%', '')

    # Removendo Caracteres
    inflation = float(value.replace(',', '.'))
    inflation_str = str(inflation).replace('.', ',') + '%' # Corrigindo Erro

    print(f'Taxa de Inflação Americana:: {inflation_str}')

Taxa de Inflação Americana:: 6,41%


In [114]:
url = 'https://ycharts.com/indicators/us_10year_government_bond_interest_rate'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

xpath = '/html/body/main/div/div[2]/div/div/div[2]'
value_elem = html.find('div', {'class':'key-stat-title'}).get_text()

if value_elem is not None:
    value = value_elem.strip()
    value = value.split(' ')[0] # Removendo Strings
    value = value.replace('%', '')

    # Removendo Caracteres
    interest_rate = float(value.replace(',', '.'))
    interest_rate_str = str(interest_rate).replace('.', ',') + '%' # Corrigindo Erro

    print(f'Taxa de Juros Livre de Risco (Rf): {interest_rate_str}')

Taxa de Juros Livre de Risco (Rf): 3,53%


In [115]:
# Set up scraper
url = 'https://www.investing.com/rates-bonds/brazil-1-year-bond-yield'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

rm_str = html.find('span', {'class': 'arial_26 inlineblock pid-24024-last'})
rm = float(rm_str.text)
rm_perc = rm/100
rm_perc_str = str(rm_perc).replace('.', ',')

print(f'Retorno Carteira de Mercado (Rm): {rm_perc_str}')

Retorno Carteira de Mercado (Rm): 0,13401


In [116]:
# Set up scraper
url = 'https://br.investing.com/economic-calendar/brazilian-cpi-410'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

inflation_str = html.find('div', {'class': 'arial_14 redFont'})
inflation = float(inflation_str.text.replace(',', '.').replace('%', ''))
inflation_perc_str = str(inflation).replace('.', ',')

print(f'Taxa de Inflação Brasil (%) : {inflation_perc_str}%')

Taxa de Inflação Brasil (%) : 5,77%


## **CAPM - Custo do Capital Próprio**

Custo de Capital Próprio

In [117]:
rf = 4.5/100
rj = rf + (beta*(rm_perc - rf))
rj_str = str(rj).replace('.', ',')

print(f'Rj (Ke): {rj_str}')

Rj (Ke): 0,1660536


Ki (Kd) - Custo da Dívida

PL / (Po+PL)

Po / (Po+PL)

**WACC**

# **Balanço Patrimonial**

In [118]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.zacks.com/stock/quote/WIRE/balance-sheet'
user_agent = {'User-agent': 'Mozilla/5.0'}

# Obtendo a página
page = requests.get(url, headers = user_agent)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Extraindo os valores da tabela
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

[]
['Strong Buy', '24.28%']
['Buy', '17.83%']
['Hold', '9.19%']
['Sell', '5.00%']
['Strong Sell', '2.16%']
['500', '10.51%']
['A']
['A']
['A']
['A']
[]
['Assets', '\xa0', '\xa0', '\xa0', '\xa0', '\xa0']
['Cash & Equivalents', '731', '439', '183', '231', '178']
['Receivables', '514', '492', '277', '227', '237']
['Notes Receivable', '0', '0', '0', '0', '0']
['Inventories', '153', '101', '92', '90', '102']
['Other Current Assets', '4', '3', '3', '2', '2']
['Total Current Assets', '1,402', '1,035', '555', '549', '519']
['Net Property & Equipment', '617', '495', '411', '333', '299']
['Investments & Advances', '0', '0', '0', '0', '0']
['Other Non-Current Assets', '0', '0', '0', '0', '0']
['Deferred Charges', '0', '0', '0', '0', '0']
['Intangibles', '0', '0', '0', '0', '0']
['Deposits & Other Assets', '0', '1', '1', '1', '0']
['Total Assets', '2,019', '1,531', '966', '883', '818']
[]
['Notes Payable', '0', '0', '0', '0', '0']
['Accounts Payable', '63', '75', '57', '41', '37']
['Current Portio

Patrimônio Líquido (PL)

In [119]:
# Encontrando o valor de Total Shareholder's Equity
total_shareholders_equity = soup.find('td', text="Total Shareholder's Equity")
value = total_shareholders_equity.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print(value_formatted)


$1,819,000,000.00
